In [253]:
#import modules for web scraping
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup

In [55]:
#get html code
url = 'https://www.boxofficemojo.com/alltime/domestic.htm'
response = requests.get(url)

In [56]:
#check if it worked
response.status_code

200

In [94]:
#make soup
page = response.text
soup_domestic = BeautifulSoup(page,"lxml")

In [95]:
print(soup_domestic.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <title>
   All Time Domestic Box Office Results
  </title>
  <meta content="all, time, domestic, box, office, result, gross, grosses, titanic, all, time, report, movie, film" name="keywords"/>
  <meta content="All time domestic box office results." name="description"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="screen" rel="stylesheet" title="no title" type="text/css"/>
  <link charset="utf-8" href="/css/mojo.css?1" media="print" rel="stylesheet" title="no title" type="text/css"/>
 </head>
 <body>
  <iframe frameborder="0" height="1" id="sis_pixel_sitewide" marginheight="0" marginwidth="0" style="display: none;" width="1">
  </iframe>
  <script>
   setTimeout(function(){
        try{
            //sis3.0 pixel
            var cacheBust = Math.random() * 10000000000000000,
                url_sis3 = '//s.amazon-adsystem.com/iu3?',
                p

In [139]:
tables=soup_domestic.find_all("table")
rows=[row for row in tables[5].find_all('tr')]
columns = rows[0]
rows = rows[1:20]

# for row in rows:
#     print(row)
#     print()

movies = {}
for row in rows:
    test_items = row.find_all('td')
    test_title = test_items[1].find('a')['href']
    movies[test_title] = [i.text for i in test_items[1:]]

columns = columns.find_all('a')
columns = [column.text for column in columns][1:]

# print(movies)

# list(movies.keys())
# import defaultdict

# movies_dict_for_df = defaultdict()
# for key in movies:
#     movies[key][0] = 

In [254]:
#clean the title with regex as an exercise
import re

pattern = re.compile(r'[a-zA-z ]+')
# for column in columns:
#     pattern.search(column).group(0)
#     print(column)

columns = [pattern.search(column).group(0) for column in columns]
columns = [column.replace('^','') for column in columns]

column_names = columns
column_names
    
# columns[0] = 'Title'
# columns[-1] = 'Year'


['Title', 'Studio', 'Lifetime Gross', 'Year']

In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

%matplotlib inline

In [171]:
#first rough df for the domestic gross figures for movies
domestic_df = pd.DataFrame(list(movies.values()), columns = column_names)
domestic_df.head()

,Title,Studio,Lifetime Gross,Year
0,Star Wars: The Force Awakens,BV,"$936,662,225",2015
1,Avatar,Fox,"$760,507,625",2009^
2,Black Panther,BV,"$700,059,566",2018
3,Avengers: Infinity War,BV,"$678,815,482",2018
4,Titanic,Par.,"$659,363,944",1997^


In [256]:
# starting to scrape ski resort data
# grab data for each relevant ski resort: total snowfall, snow days, average/max base depth, average/max summit depth,
# biggest snowfall days
# injuries per year

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Users/dynogravelso/Documents/GitHub/luther/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [322]:
# go to base page
url = 'https://www.skiresort.info/ski-resorts/north-america/'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [323]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [324]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
#navigate to ski lifts
# ski_lift_selector = '//a[contains(text(), "Ski lifts")]'
# driver.find_element_by_xpath(ski_lift_selector).click()

In [325]:
#find all resorts here
resort_soup = soup.find(id = 'resortList')
sub_list = resort_soup.find_all('a', {'class': 'h3'})    
resorts = [resort.text for resort in sub_list]
all_resorts = []
all_resorts += resorts

In [327]:
def scrape_once(url, all_resorts):
    driver.get(url)
    time.sleep(15)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    resort_soup = soup.find(id = 'resortList')
    sub_list = resort_soup.find_all('a', {'class': 'h3'})    
    resorts = [resort.text for resort in sub_list]
    all_resorts += resorts
    return all_resorts
    

In [328]:
#navigate to next page and find 
for i in range(2,6):
    url_new = 'https://www.skiresort.info/ski-resorts/north-america/page/' + str(i) + '/'
    all_resorts = scrape_once(url_new, all_resorts)

In [331]:
#clean up resort names
all_resorts = [resort.strip() for resort in all_resorts]
all_resorts


['Whistler Blackcomb',
 'Vail',
 'Telluride',
 'Beaver Creek',
 'Mammoth Mountain',
 'Snowmass',
 'Breckenridge',
 'Sun Peaks',
 'Mont Tremblant',
 'Lake Louise',
 'Keystone',
 'Big White',
 'Stowe',
 'Heavenly',
 'Winter Park Resort',
 'Silver Star',
 'Fernie',
 'Squaw Valley',
 'Sunshine Village',
 'Panorama',
 'Park City',
 'Steamboat',
 'Kicking Horse – Golden',
 'Big Sky Resort',
 'Aspen Highlands',
 'Marmot Basin – Jasper',
 'Kimberley',
 'Alpine Meadows',
 'Killington',
 'Copper Mountain',
 'Alta',
 'Revelstoke Mountain Resort',
 'Nakiska',
 'Buttermilk Mountain',
 'Jackson Hole',
 'Crested Butte',
 'Powder Mountain',
 'Mt. Bachelor',
 'Whiteface – Lake Placid',
 'Mont Saint-Sauveur',
 'Deer Valley',
 'Northstar California Resort',
 'Yellowstone Club',
 'Sugarloaf',
 'Castle Mountain',
 'Snowbird',
 'Bald Mountain – Sun Valley',
 'Arapahoe Basin',
 'Mt. Hood Meadows',
 'Stratton',
 'Schweitzer Mountain Resort',
 'Red Mountain Resort – Rossland',
 'Cypress Mountain',
 'Mt. Rose',

In [204]:
driver.close()